In [ ]:
# import time
# start_time = time.time()
# total_time = (4 * 60 + 45) * 60

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from transformers import set_seed
set_seed(42)

In [ ]:
def get_environment() -> str:
    if os.getenv("KAGGLE_KERNEL_RUN_TYPE") != None:
        return "kaggle"
    elif os.getenv("COLAB_RELEASE_TAG") != None:
        return "colab"
    else:
        return "local"


ENV = get_environment()
if ENV == "kaggle":
    from jeffdshen.aimo2.server import load_qwq, MODELS
    from jeffdshen.aimo2.predict import MetaLLM, SYSTEM_PARAMS
    from jeffdshen.aimo2.config import get_validation_data
elif ENV == "local":
    from .server import load_qwq, MODELS
    from .predict import MetaLLM, SYSTEM_PARAMS
    from .config import get_validation_data
elif ENV == "colab":
    raise NotImplementedError("Not yet implemented")
else:
    raise ValueError("Unknown environment")

In [1]:
data = get_validation_data()

In [ ]:
llm = load_qwq(MODELS["qwq-32b-preview-awq"])

In [ ]:
SYSTEMS = [
    "v1",
    "v2",
    "v3",
    "v4",
    "v5",
    "default_v1",
    "default_v2",
    "default_v3",
    "think_v1",
    "think_v2",
]

In [ ]:
with open("request_log.jsonl", "w") as f:
    meta_llm = MetaLLM(
        llm=llm,
        system_params=[SYSTEM_PARAMS[x] for x in SYSTEMS],
        question_log=f,
        correct_answers=data.correct_answers,
    )
    meta_llm.predict(data.df.get_column("id"), data.df.get_column("problem"))